# Buscador de tendencias de nombres en Estados Unidos (1910-2020)

## Carga de datos

In [1]:
# librerias a utilizar
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ipywidgets as widgets

# cliente bigquery para consultar database
client = bigquery.Client()

# Cargo la tabla de nombres que me interesa
dataset_ref = client.dataset('usa_names', project='bigquery-public-data')
table_ref = dataset_ref.table('usa_1910_current')
names_table = client.get_table(table_ref)
                               
# Cabecera de la tabla, para inspeccionar datos
# client.list_rows(names_table, max_results=3).to_dataframe()

# Cuota de lectura de BigQuery
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

## Métodos útiles

In [3]:
def get_top_names_decade(decade):
    '''  Devuelve un DataFrame con el Top 5 (masculino y femenino) 
    de nombres mas recurrentes de la década indicada
    
    @args:
        decade: década sobre la que interesa conocer la tendencia de nombres
    
    >> get_top_names_decade(1910)
    DataFrame(name, gender, decade_number, decade_ranking)   
    '''
    
    top_names_decade_query =  '''
    SELECT name AS Nombre, 
           gender AS Genero,
           SUM(number) AS NumUsosDecada, 
           RANK() OVER (
               PARTITION BY gender
               ORDER BY SUM(number) DESC
           ) AS RankingDecada
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE year >= @decade AND year < (@decade + 10)
    GROUP BY name, gender
    ORDER BY RankingDecada ASC
    LIMIT 10
    '''  
    # para pasaje de parametros variables a la query
    safe_config.query_parameters = [bigquery.ScalarQueryParameter("decade", "INT64", decade)]
    
    top_names_decade_query_job = client.query(top_names_decade_query, job_config=safe_config)
    top_names = top_names_decade_query_job.to_dataframe()
    return top_names

In [4]:
# No pude resolver que utilice los nombres de top_names_decade
# y aca solo consulte su evolucion
# Asi que reutilizo codigo y hago aca por completo
def get_top_names_decade_evolution(decade):
    ''' Devuelve un Dataframe con la evolucion en el tiempo 
    de los nombres más recurrentes de la década indicada.
    
    @args:
        - decade: década sobre la cual consultar los nombres más recurrentes 
          (de los cuales luego se analiza su evolcuión en toda la historia)
    
    >> get_top_names_decade_evolution(1930)
    Dataframe(year, name, gender, year_number)
    '''
    
    top_names_decade_evolution_query =  '''
    WITH top_names_decade AS (
        SELECT name, 
               gender,
               SUM(number) AS decade_number, 
               RANK() OVER (
                   PARTITION BY gender
                   ORDER BY SUM(number) DESC
               ) AS decade_ranking
        FROM `bigquery-public-data.usa_names.usa_1910_current`
        WHERE year >= @decade AND year < (@decade + 10)
        GROUP BY name, gender
        ORDER BY decade_ranking ASC
        LIMIT 10
    )

    SELECT n.year AS Anio, n.name AS Nombre, n.gender AS Genero, SUM(n.number) AS NumUsosAnio    
    FROM `bigquery-public-data.usa_names.usa_1910_current` AS n
    RIGHT JOIN top_names_decade AS tnd
        ON tnd.name = n.name AND tnd.gender = n.gender
    GROUP BY n.year, n.name, n.gender
    '''  
    
    # para pasaje de parametros variables a la query
    safe_config.query_parameters = [bigquery.ScalarQueryParameter("decade", "INT64", decade)]
    
    top_names_decade_evolution_query_job = client.query(top_names_decade_evolution_query, job_config=safe_config)
    top_names_decade_evolution = top_names_decade_evolution_query_job.to_dataframe()
    return top_names_decade_evolution    

In [84]:
def get_names_evolution():
    names_evolution_query = '''
    SELECT year, 
           name,
           SUM(number) AS year_number,
           RANK() OVER (
               PARTITION BY year
               ORDER BY SUM(number) DESC
           ) AS year_ranking 
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY year, name
    ORDER BY year, year_number DESC
    '''
    
    names_evolution_query_job = client.query(names_evolution_query, job_config=safe_config)
    names_evolution = names_evolution_query_job.to_dataframe()
    return names_evolution


def get_selected_name_evolution(search_name):
    # capitalizo -> porque asi esta en el Dataset
    search_name = search_name.capitalize()
    names_evolution = get_names_evolution()
    selected_name_evolution = names_evolution[names_evolution['name'] == search_name]
       
    # Pruebo 'merge' para que el resultado tenga todos los años,
    # aunque el nombre no tenga data de algunos
    # UPD: al pedo, el ploteo no muestra los NaN (no pone 0, como pense)
    # years = pd.Series(range(1910, 2021))
    # years_df = pd.DataFrame(data={'year': years})
    # result = pd.merge(years_df, selected_name_evolution, on='year', how='left')
    return result

In [6]:
# ToDo: resolver to_decades()
# Por ahora, harcodeo

def get_data_decades():
    ''' Devuelve los años sobre los que hay información disponible en el Dataset USA_Names.
    En particular, en forma de "décadas".
    
    @args:
    
    >> get_data_decades()
    [1910, 1920]
    '''
    
    years_query = '''
    SELECT year
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY year
    '''
    years_query_job = client.query(years_query, job_config=safe_config)
    years = years_query_job.to_dataframe()
    
    data_decades = to_decades(years.year.to_list())
    return data_decades

def to_decades(years):
    ''' Devuelve un listado de decadas (1910, 1920, ...) a partir de un listado de años recibidos
    [1911,1912,1914, 1925,...]
    
    @args:
        years: lista de años sobre los que se calculara las decadas a las que hace alusión
    
    >> to_decades([1911,1912,1914, 1925,...])
    [1910, 1920]
    '''
    decades = [1910, 1920, 1930,1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
    return decades

print(get_data_decades())

[1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]


In [66]:
def get_data_names():
    # total de nombres distintos
    names_query = '''
    SELECT DISTINCT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    ORDER BY name ASC
    '''

    names_query_job = client.query(names_query, job_config=safe_config)
    names = names_query_job.to_dataframe()
    
    data_names = list(names.name.unique())
    return data_names

# si el nombre existe dentro del set de datos
def exists_name(name):
    # capitalizo -> porque asi esta en el Dataset
    name = name.capitalize()
    return name in data_names

## Buscador por década. Nombres Tendencia
Este buscador permite encontrar los 5 nombres femeninos y masculinos más utilizados en la década indicada, y su evolución a lo largo del tiempo.

In [23]:
dropdown_decade = widgets.Dropdown(options=get_data_decades())

#captura el valor de la salida -> Lo que se dispara al seleccionar un item del dropdwn
output_table = widgets.Output()
output_plot = widgets.Output()

# handler al seleccionar item del dropdwn
def dropdown_decade_eventhandler(change):
    output_table.clear_output()
    output_plot.clear_output()
    
    top_names_decade = get_top_names_decade(change.new)
    top_names_decade_evolution = get_top_names_decade_evolution(change.new)
    
    with output_table:
        display(top_names_decade)
    
    with output_plot:
        plt.figure(figsize=(8,4))
        sns.set_style('whitegrid')
        sns.lineplot(x=top_names_decade_evolution.Anio, y=top_names_decade_evolution.NumUsosAnio, hue=top_names_decade_evolution.Nombre)
        plt.show()

# queda a la espera de un evento en el dropdown        
dropdown_decade.observe(dropdown_decade_eventhandler, names='value')
display(dropdown_decade)

Dropdown(options=(1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020), value=1910)

In [24]:
display(output_table)
display(output_plot)

Output()

Output()

## Buscador por Nombre
Este nos permite ingresar un nombre y conocer en qué decadas fue tendencia (Top 10), y si no existiera ningún momento en que lo haya sido, indicará cuál fue el año en que más se utilizo y su posición en comparación con el resto.
Opcion: que tmb muestre su evolución en el tiempo.
Opcion: link a google con el nombre elegido y la decada en que fue tendencia, para tratar de encontrar razones por las que fue trending. Ejemplo: https://www.google.com/search?q=peter+1974

In [10]:
data_names = get_data_names()

In [87]:
textbox_name = widgets.Text(placeholder='Escriba un nombre')
btn = widgets.Button(description='Buscar')

#captura el valor de la salida -> Lo que se dispara al seleccionar un item del dropdwn
output_name_evolution_plot = widgets.Output()

# handler al seleccionar item del dropdwn
def btn_eventhandler(obj):
    output_name_evolution_plot.clear_output()
    search_name = textbox_name.value
    
    with output_name_evolution_plot:
        if not exists_name(search_name): 
            print('El nombre {} no se encuentra en la lista'.format(search_name.capitalize()))
            return
        
        btn.description = 'Buscando...'
        selected_name_evolution = get_selected_name_evolution(search_name) 
        
        # graficos de cantidad de usos y ranking
        fig, axes = plt.subplots(2,1, sharex=True, figsize=(8,8))
        sns.set_style('whitegrid')
        sns.lineplot(ax=axes[0], x=selected_name_evolution.year, y=selected_name_evolution.year_number)
        sns.lineplot(ax=axes[1], x=selected_name_evolution.year, y=selected_name_evolution.year_ranking)    
        
        plt.setp(axes[-1], xlabel='Año')
        axes[0].set_title('Cantidad de usos a lo largo del tiempo')
        axes[1].set_title('Ranking a lo largo del tiempo (usos comparado con el resto)')
        axes[0].set_ylabel('Cantidad de usos')
        axes[1].set_ylabel('Ranking')
        
        plt.show()
        btn.description = 'Buscar'

# queda a la espera de un evento en el dropdown        
btn.on_click(btn_eventhandler)
display(textbox_name)
display(btn)
display(output_name_evolution_plot)

Text(value='', placeholder='Escriba un nombre')

Button(description='Buscar', style=ButtonStyle())

Output()

**ToDo:** agregar un pequeño resumen *highlights* del nombre, indicando el año en que fue más utilizado y su puesto en el ranking.